In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_log_error, r2_score

from xgboost import XGBRegressor

In [ ]:
df_train = pd.read_csv("sample_data/train.csv")
X_finaltest = pd.read_csv("sample_data/test.csv")

In [ ]:
X_finaltest

In [ ]:
X

In [ ]:
X = df_train.drop(["Survived"], axis=1)
X = X.drop(["Name"], axis=1)
X_finaltest = X_finaltest.drop(["Name"], axis=1)

In [ ]:
y = df_train['Survived']
y

In [ ]:
encoder = OrdinalEncoder()
#X['Sex'] = encoder.fit_transform(X['Sex'].values.reshape(-1, 1))
#X['Sex']

In [ ]:
import re
i=0
for tick in X_finaltest.Ticket:
  X_finaltest.loc[i,('Ticket')] = re.sub('[^0-9]','', X_finaltest.loc[i,('Ticket')])
  i+=1 
i=0  
for tick in X.Ticket:
  X.loc[i,('Ticket')] = re.sub('[^0-9]','', X.loc[i,('Ticket')])
  i+=1  

In [ ]:
X = X.astype({'Ticket': 'float'})
#X_test = X_test.astype({'Ticket': 'float'})

In [ ]:
X.Ticket

In [ ]:
features = ['Sex','Cabin','Embarked','Ticket']

In [ ]:
for feature in features:
  if X[feature].dtype == 'object':
    X[feature] = encoder.fit_transform(X[feature].values.reshape(-1, 1))
    X_finaltest[feature] = encoder.fit_transform(X_finaltest[feature].values.reshape(-1, 1))

In [ ]:
imp = SimpleImputer(strategy="mean")
X[X.columns] = imp.fit_transform(X[X.columns])
X_finaltest[X_finaltest.columns] = imp.fit_transform(X_finaltest[X_finaltest.columns])

In [ ]:
scaler = RobustScaler()
for feature in X.columns:
  X[feature] = scaler.fit_transform(X[feature].values.reshape(-1, 1))
  if feature == 'Survived':
    break
  X_finaltest[feature] = scaler.fit_transform(X_finaltest[feature].values.reshape(-1, 1))

In [ ]:
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
X['Cabin']

0      76.0
1      81.0
2      76.0
3      55.0
4      76.0
       ... 
886    76.0
887    30.0
888    76.0
889    60.0
890    76.0
Name: Cabin, Length: 891, dtype: float64

In [ ]:
sns.displot(df_train['Embarked'])

In [ ]:
total = X.isnull().sum().sort_values(ascending=False)
percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

missing_data=missing_data.head(35)
f, ax = plt.subplots(figsize=(16, 8))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.title('Percent missing data by feature', fontsize=15)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [ ]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=25)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)

In [ ]:
y_finaltest = logreg.predict(X_finaltest)

In [ ]:
from sklearn.metrics import accuracy_score

score =accuracy_score(y_test,y_finaltest)
score

1.0

In [ ]:
len(y_finaltest)

418

In [ ]:
submitionDF = pd.DataFrame({'Survived':y_finaltest})

In [ ]:
submitionDF['PassengerId'] = submitionDF.index + 892

In [ ]:
submitionDF.to_csv('titanic2.csv', index=False)

In [ ]:
submitionDF

,Survived,PassengerId
0,0,892
1,0,893
2,0,894
3,0,895
4,1,896
...,...,...
413,0,1305
414,1,1306
415,0,1307
416,0,1308


**Tensorflow model**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import seaborn as sns

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [ ]:
features = X.columns

In [ ]:
feature_columns = []

for feat in features:
  feat = tf.feature_column.numeric_column(feat)
  feature_columns.append(feat)



# Convert the list of feature columns into a layer that will later be fed into
# the model. 
my_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
def log_loss(y_pred, y):
  # Compute the log loss function
  ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(ce)

In [ ]:
class LogisticRegression(tf.Module):

  def __init__(self):
    self.built = False

  def __call__(self, x, train=True):
    # Initialize the model parameters on the first call
    if not self.built:
      # Randomly generate the weights and the bias term
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1], seed=22)
      rand_b = tf.random.uniform(shape=[], seed=22)
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    # Compute the model output
    z = tf.add(tf.matmul(x, self.w), self.b)
    z = tf.squeeze(z, axis=1)
    if train:
      return z
    return tf.sigmoid(z)

In [ ]:
log_reg = LogisticRegression()

In [ ]:
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
y_pred = log_reg(X[:5], train=False)
y_pred.numpy()

InvalidArgumentError: ignored

In [ ]:
X.dtypes

PassengerId    float64
Pclass         float64
Sex            float64
Age            float64
SibSp          float64
Parch          float64
Ticket         float64
Fare           float64
Cabin          float64
Embarked       float64
dtype: object